In [13]:
!nvidia-smi
!pip install transformers datasets accelerate gradio -q


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Sun Nov  2 16:55:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   61C    P0             30W /   70W |    2281MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [14]:
import pandas as pd
import numpy as np
from datasets import Dataset
import re


In [15]:
df = pd.read_csv("/kaggle/input/3a2mext/3A2M_EXTENDED.csv")
print(df.columns)
df.head()


Index(['title', 'NER', 'Extended_NER', 'genre', 'label', 'directions'], dtype='object')


,title,NER,Extended_NER,genre,label,directions
0,\t Arugula Pomegranate Salad,"[""baby spinach"", ""baby arugula"", ""pomegranate ...","['alfalfa sprouts', 'baby spinach', 'baby arug...",vegetables,4,"[""Toss together spinach and arugula, then plac..."
1,\t Black Bean And Turkey Chili,"[""olive oil"", ""yellow onion"", ""garlic"", ""groun...","['one', 'yellow onion', 'tomato paste', 'about...",sides,8,"[""Dice the onion and mince the garlic. Add the..."
2,\t Finger Lickin' Tofu Nuggets,"[""extra firm"", ""almond flour"", ""nutritional ye...","['extra firm', '2', 'coconut oil', 'almond flo...",nonveg,3,"[""Wrap the tofu in a clean tea towel and press..."
3,\t Jerk Beef Stew With Carrots And Tomatoes,"[""olive oil"", ""boneless beef chuck"", ""onion"", ...","['boneless beef chuck', '2', 'Saute', 'onion',...",vegetables,4,"[""Preheat oven to 350 degrees F."", ""Heat the o..."
4,\t Pomegranate Couscous Salad,"[""pomegranate arils"", ""whole wheat couscous"", ...","['whole wheat couscous', '10 minutes', 'lemon ...",vegetables,4,"[""Place couscous in a bowl with 11/2 cups of h..."


In [16]:
# ====================================================
# 1. Basic Cleaning and Column Normalization
# ====================================================
df.columns = df.columns.str.strip().str.lower()

# Remove completely empty rows
df.dropna(how='all', inplace=True)

# ====================================================
# 2. Drop duplicate rows safely (Option 1)
# ====================================================
df_str = df.copy()
for col in ["ner", "extended_ner", "directions"]:
    if col in df_str.columns:
        df_str[col] = df_str[col].astype(str)  # Convert lists to strings for deduplication

df = df.loc[~df_str.duplicated()].copy()
print(f"✅ Duplicates removed, current shape: {df.shape}")

# ====================================================
# 3. Clean list-like columns safely
# ====================================================
def clean_list_column(col):
    cleaned = []
    for val in col:
        # Case 1: Already a list/tuple — keep it as is
        if isinstance(val, (list, tuple)):
            cleaned.append(list(val))
            continue

        # Case 2: Empty or NaN
        if pd.isna(val):
            cleaned.append([])
            continue

        # Case 3: Try parsing JSON-like strings
        try:
            parsed = json.loads(str(val).replace("'", '"'))
            if isinstance(parsed, list):
                cleaned.append(parsed)
            else:
                cleaned.append([str(parsed)])
        except:
            # Case 4: Fallback to comma-split
            cleaned.append([x.strip() for x in str(val).split(",")])
    return cleaned


for col in ["ner", "extended_ner", "directions"]:
    if col in df.columns:
        df[col] = clean_list_column(df[col])

# Drop rows missing essential data
if "title" in df.columns and "directions" in df.columns:
    df.dropna(subset=["title", "directions"], inplace=True)

print(f"✅ After cleaning list-like columns: {df.shape}")

# ====================================================
# 4. Skip Genre Normalization (as requested)
# ====================================================
if "genre" not in df.columns:
    df["genre"] = "unknown"

# ====================================================
# 5. Add Derived Columns
# ====================================================
df["ingredient_count"] = df["ner"].apply(lambda x: len(x) if isinstance(x, list) else 0)
df["direction_length"] = df["directions"].apply(lambda x: len(x) if isinstance(x, list) else 0)

# Remove rows with no ingredients or directions
df = df[(df["ingredient_count"] > 0) & (df["direction_length"] > 0)]

# Drop duplicates based on title + genre
df.drop_duplicates(subset=["title", "genre"], inplace=True)

# Reset index
df.reset_index(drop=True, inplace=True)

print(f"✅ Final cleaned dataset shape: {df.shape}")

# ====================================================
# 6. Display Random Samples
# ====================================================
print("\n=== Random Sample of Cleaned Recipes ===\n")
sample_df = df.sample(min(10, len(df)), random_state=42)[["title", "genre", "ner", "directions"]]

for i, row in sample_df.iterrows():
    print(f"Recipe {i+1}: {row['title']}")
    print(f"Genre: {row['genre']}")
    print(f"Ingredients ({len(row['ner'])}): {', '.join(row['ner'][:10])}...")
    print(f"First 2 Steps: {row['directions'][:2]}")
    print("-" * 100)

# ====================================================
# 7. Summary Stats
# ====================================================
print("\n=== Dataset Statistics ===")
print(f"Average number of ingredients: {df['ingredient_count'].mean():.2f}")
print(f"Average number of directions: {df['direction_length'].mean():.2f}")
print(f"Unique genres: {df['genre'].nunique()} -> {sorted(df['genre'].unique().tolist())}")

# ====================================================
# 8. Save Cleaned CSV
# ====================================================
output_path = "cleaned_recipes.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f"\n✅ Cleaned dataset saved as: {output_path}")


✅ Duplicates removed, current shape: (2230998, 6)
✅ After cleaning list-like columns: (2230997, 6)
✅ Final cleaned dataset shape: (1314993, 8)

=== Random Sample of Cleaned Recipes ===

Recipe 622937: Honey-Dijon Salad with Shrimp
Genre: vegetables
Ingredients (11): ["torn romaine lettuce leaves", "shrimp", "mushrooms", "carrots", "egg substitute", "corn oil", "white wine vinegar", "Mustard", "honey", "croutons"...
First 2 Steps: ['["Mix lettuce', 'shrimp']
----------------------------------------------------------------------------------------------------
Recipe 666671: Kaessuppe
Genre: fusion
Ingredients (7): ["butter", "flour", "emmentaler cheese", "beef stock", "egg yolks", "sugar", "bread"]...
First 2 Steps: ['["Melt the butter and stir in the flour."', '"Add meat broth."']
----------------------------------------------------------------------------------------------------
Recipe 284614: Chimichurri Muy Perfecto
Genre: fusion
Ingredients (10): ["flat-leaf Italian parsley", "olive 

In [17]:
import pandas as pd
import ast
import re

# --- Robust cleaning function ---
def clean_recipe(row):
    # --- Helper to remove brackets, quotes, and fix unicode ---
    def clean_text(text):
        if not text:
            return ""
        text = str(text)
        # Fix degree/Fahrenheit symbols
        text = text.replace("\\u00b0", "°")  # Unicode ° symbol
        text = text.replace("\\u00b0F", "°F")
        text = text.replace("\\u00b0C", "°C")
        # Remove all [ ] { } ' " ` and extra spaces
        text = re.sub(r"[\[\]{}'\"`]", "", text)
        text = re.sub(r"\s+", " ", text)
        return text.strip()

    # --- Ingredients ---
    ner_val = row.get("ner", [])
    if isinstance(ner_val, str):
        try:
            ner_val = ast.literal_eval(ner_val)
        except:
            ner_val = [ner_val]
    ingredients = ", ".join([clean_text(i) for i in ner_val if str(i).strip()])

    # --- Directions ---
    dir_val = row.get("directions", [])
    if isinstance(dir_val, str):
        try:
            dir_val = ast.literal_eval(dir_val)
        except:
            dir_val = [dir_val]
    # Flatten nested lists
    flat_dirs = []
    for d in dir_val:
        if isinstance(d, (list, tuple)):
            flat_dirs.extend(d)
        else:
            flat_dirs.append(d)
    directions = " ".join([clean_text(d) for d in flat_dirs if str(d).strip()])

    # --- Title ---
    title = clean_text(row.get("title", ""))

    # --- Compose final recipe ---
    return f"<|startofrecipe|> Title: {title}\nIngredients: {ingredients}\nDirections: {directions} <|endofrecipe|>"

# --- Apply cleaning ---
df['text'] = df.apply(clean_recipe, axis=1)

# --- Remove duplicates ---
df.drop_duplicates(subset='text', inplace=True)

# --- Check remaining brackets ---
remaining_brackets = df['text'].str.contains(r'[\[\]]').sum()
print(f"Remaining brackets: {remaining_brackets}")  # Should be 0

# --- Show first 15 examples ---
for i in range(min(15, len(df))):
    print(f"Example {i+1}:\n{df['text'].iloc[i]}\n{'-'*80}")

# --- Save cleaned dataset to CSV ---
output_path = "cleaned_recipes_final.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f"\n✅ Cleaned dataset saved as: {output_path}")


Remaining brackets: 0
Example 1:
<|startofrecipe|> Title: Arugula Pomegranate Salad
Ingredients: baby spinach, baby arugula, pomegranate arils, persimmon, alfalfa sprouts
Directions: Toss together spinach and arugula then place in your serving bowl. Remove the stem and leaves of the persimmon then slice into thin wedges. Arrange the persimmon on top of the spinach and arugula. Garnish with pomegranate arils and alfalfa sprouts. <|endofrecipe|>
--------------------------------------------------------------------------------
Example 2:
<|startofrecipe|> Title: Black Bean And Turkey Chili
Ingredients: olive oil, yellow onion, garlic, ground turkey, black beans, tomatoes, tomato paste, chili powder, cumin, paprika, oregano, salt
Directions: Dice the onion and mince the garlic. Add the onion and garlic to a large stock pot with one tablespoon of olive oil and cook over medium-low heat just until softened. Add the ground turkey to the pot and continue to saute until the turkey is cooked thro

In [18]:
# Check if any brackets remain
df['text'].str.contains(r'[\[\]]').sum()  # Should be 0


0

In [19]:
# one-time installs (run in a notebook cell)
!pip install -q transformers datasets accelerate bitsandbytes peft evaluate sentencepiece gradio


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [20]:
# ==========================================================
#  DISTILGPT2 RECIPE FINE-TUNING (LoRA + 4-bit + Tokenizer Integration)
# ==========================================================
import os, gc, random
import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, TaskType

# ---------------- CONFIG ----------------
MODEL_NAME = "distilgpt2"      # smaller + faster than GPT-2
CLEANED_CSV = "cleaned_recipes_final.csv"
OUTPUT_DIR = "/kaggle/working/gpt2_recipe_lora"
NUM_EPOCHS = 3                 # 3–5 sufficient per assignment
PER_DEVICE_BATCH = 2
GRAD_ACCUM = 8
MAX_LENGTH = 512
SEED = 42
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f">>> Device: {DEVICE}")

# ---------------- REPRODUCIBILITY ----------------
random.seed(SEED)
torch.manual_seed(SEED)
if DEVICE == "cuda":
    torch.cuda.manual_seed_all(SEED)

# ---------------- LOAD DATA ----------------
df = pd.read_csv(CLEANED_CSV, encoding="utf-8-sig")
df = df[["text"]].dropna().reset_index(drop=True)
print("Loaded samples:", len(df))

dataset = Dataset.from_pandas(df)

# ---------------- TOKENIZER ----------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
special_tokens = ["<|startofrecipe|>", "<|endofrecipe|>"]
tokenizer.add_special_tokens({"additional_special_tokens": special_tokens})
tokenizer.pad_token = tokenizer.eos_token  # GPT2 has no pad token

def tokenize_batch(examples):
    return tokenizer(examples["text"], truncation=True, max_length=MAX_LENGTH)

print("Tokenizing dataset...")
dataset = dataset.map(tokenize_batch, batched=True, remove_columns=["text"], desc="Tokenizing")
dataset = dataset.train_test_split(test_size=0.1, seed=SEED)
train_ds = dataset["train"]
eval_ds  = dataset["test"]
print(f"Train size: {len(train_ds)}  |  Eval size: {len(eval_ds)}")


>>> Device: cuda
Loaded samples: 1314980
Tokenizing dataset...


Tokenizing:   0%|          | 0/1314980 [00:00<?, ? examples/s]

Train size: 1183482  |  Eval size: 131498


In [27]:
load_4bit = torch.cuda.is_available()
if load_4bit:
    print(">>> Using 4-bit quantization for efficiency")
    from transformers import BitsAndBytesConfig

    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=quant_config,
        device_map={"": torch.cuda.current_device()}  # FIX: ensures model is on correct GPU
    )
else:
    print(">>> Loading model normally (no 4-bit support detected)")
    model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(DEVICE)

# Resize embeddings for special tokens
try:
    model.resize_token_embeddings(len(tokenizer), mean_resizing=False)
except TypeError:
    model.resize_token_embeddings(len(tokenizer))

# ---------------- APPLY LORA ----------------
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["c_attn", "q_proj", "v_proj"]
)
model = get_peft_model(model, peft_config)

# Freeze non-LoRA params
for n, p in model.named_parameters():
    if "lora" not in n:
        p.requires_grad = False

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f">>> Trainable params: {trainable_params:,} / {total_params:,}")

# ---------------- DATA COLLATOR ----------------
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# ---------------- TRAINING ARGS ----------------
try:
    training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        num_train_epochs=NUM_EPOCHS,
        per_device_train_batch_size=PER_DEVICE_BATCH,
        per_device_eval_batch_size=PER_DEVICE_BATCH,
        gradient_accumulation_steps=GRAD_ACCUM,
        fp16=True,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="steps",
        logging_steps=50,
        learning_rate=5e-5,
        weight_decay=0.01,
        warmup_steps=50,
        save_total_limit=2,
        report_to="none",
        dataloader_num_workers=2,
        optim="paged_adamw_32bit"
    )
    print(">>> TrainingArguments initialized successfully.")
except TypeError:
    print(">>> Transformer version fallback detected.")
    training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        num_train_epochs=NUM_EPOCHS,
        per_device_train_batch_size=PER_DEVICE_BATCH,
        gradient_accumulation_steps=GRAD_ACCUM,
        fp16=True,
        logging_steps=50,
        save_steps=500,
        learning_rate=5e-5,
        weight_decay=0.01,
        warmup_steps=50,
        report_to="none"
    )

# ---------------- TRAINER ----------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    data_collator=data_collator,
    tokenizer=tokenizer
)

# ---------------- TRAINING ----------------
print("\n>>> Starting fine-tuning for", NUM_EPOCHS, "epochs...")
trainer.train()
print(">>> Training complete ✅")

# ---------------- SAVE MODEL ----------------
trainer.save_model(os.path.join(OUTPUT_DIR, "final_checkpoint"))
model.save_pretrained(os.path.join(OUTPUT_DIR, "lora_adapter"))
tokenizer.save_pretrained(OUTPUT_DIR)
print(">>> Model + tokenizer saved to", OUTPUT_DIR)

# ---------------- GENERATE SAMPLE OUTPUT ----------------
print("\n>>> Generating sample recipe for qualitative evaluation...")
prompt = "<|startofrecipe|> Title: Creamy Mushroom Pasta\nIngredients: mushroom, cream, garlic, pasta\nDirections:"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)  # FIX: use model.device

generation = model.generate(
    input_ids,
    max_length=200,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.8,
    num_return_sequences=1
)

print("\n=== SAMPLE GENERATION ===\n")
print(tokenizer.decode(generation[0], skip_special_tokens=False))

>>> Using 4-bit quantization for efficiency


/tmp/ipykernel_37/3923081453.py:90: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


>>> Trainable params: 147,456 / 60,827,904
>>> Transformer version fallback detected.

>>> Starting fine-tuning for 3 epochs...


Step,Training Loss
50,27.257300


KeyboardInterrupt: 

In [24]:
from IPython.display import FileLink, FileLinks

# Download the entire folder
FileLinks("/kaggle/working/gpt2_recipe_lora/checkpoint-2000")


/kaggle/working/gpt2_recipe_lora/checkpoint-2000/
  scheduler.pt
  added_tokens.json
  optimizer.pt
  tokenizer_config.json
  vocab.json
  rng_state.pth
  adapter_model.safetensors
  adapter_config.json
  trainer_state.json
  scaler.pt
  tokenizer.json
  training_args.bin
  README.md
  merges.txt
  special_tokens_map.json

In [26]:
 import shutil

checkpoint_path = "/kaggle/working/gpt2_recipe_lora/checkpoint-2000"
zip_path = "/kaggle/working/gpt2_recipe_lora/checkpoint-2000.zip"

shutil.make_archive(checkpoint_path, 'zip', checkpoint_path)
print("✅ Checkpoint zipped at:", zip_path)

# Make it downloadable
FileLink(zip_path)


✅ Checkpoint zipped at: /kaggle/working/gpt2_recipe_lora/checkpoint-2000.zip


/kaggle/working/gpt2_recipe_lora/checkpoint-2000.zip

In [28]:
# ==========================================================
#  GENERATE SAMPLE RECIPES FROM LoRA + 4-BIT CHECKPOINT (FIXED)
# ==========================================================
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# ---------- PATHS ----------
BASE_MODEL = "distilgpt2"
CHECKPOINT_DIR = "/kaggle/working/gpt2_recipe_lora/checkpoint-2000"

# ---------- DEVICE ----------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f">>> Using device: {DEVICE}")

# ---------- LOAD TOKENIZER ----------
tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT_DIR)
tokenizer.pad_token = tokenizer.eos_token

# ---------- LOAD BASE MODEL ----------
print(">>> Loading base model...")
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

# 🚨 Resize embeddings to match tokenizer (important fix)
base_model.resize_token_embeddings(len(tokenizer))

# ---------- LOAD LoRA ADAPTER ----------
print(">>> Loading fine-tuned LoRA adapter...")
model = PeftModel.from_pretrained(base_model, CHECKPOINT_DIR)
model = model.to(DEVICE)
model.eval()

# ---------- SAMPLE PROMPTS ----------
prompts = [
    "<|startofrecipe|> Title: Creamy Mushroom Pasta\nIngredients: mushroom, cream, garlic, pasta\nDirections:",
    "<|startofrecipe|> Title: Spicy Chicken Curry\nIngredients: chicken, chili, onion, tomato, garlic\nDirections:",
    "<|startofrecipe|> Title: Chocolate Chip Cookies\nIngredients: flour, sugar, butter, chocolate chips\nDirections:",
    "<|startofrecipe|> Title: Fresh Lemonade\nIngredients: lemon, sugar, water, ice\nDirections:"
]

# ---------- GENERATION SETTINGS ----------
gen_kwargs = dict(
    max_new_tokens=150,
    do_sample=True,
    temperature=0.8,
    top_p=0.95,
    top_k=50,
    pad_token_id=tokenizer.eos_token_id,
)

# ---------- GENERATE ----------
print("\n>>> Generating sample recipes...\n")
for prompt in prompts:
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(DEVICE)
    outputs = model.generate(input_ids, **gen_kwargs)
    print("=" * 70)
    print(f"Prompt:\n{prompt}\n")
    print(tokenizer.decode(outputs[0], skip_special_tokens=False))
    print("=" * 70, "\n")


>>> Using device: cuda
>>> Loading base model...


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1803: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


>>> Loading fine-tuned LoRA adapter...

>>> Generating sample recipes...

Prompt:
<|startofrecipe|> Title: Creamy Mushroom Pasta
Ingredients: mushroom, cream, garlic, pasta
Directions:

<|startofrecipe|> Title: Creamy Mushroom Pasta
Ingredients: mushroom, cream, garlic, pasta
Directions: Preheat oven to 350 degrees. In a large skillet over medium-high heat add mushrooms and garlic and cook until browned. In a medium heat combine mushrooms and garlic and cook until golden browned. Add mushrooms and cook for 5 minutes until mushrooms are golden browned and on top of mushrooms. Add mushrooms and cook until mushrooms are golden browned and on top of mushrooms. Add mushrooms and cook for 5 minutes. Add mushrooms and cook for 5 minutes. Add mushrooms and cook for 5 minutes. Add mushrooms and cook for 5 minutes. Add mushrooms and cook for 5 minutes. In a large skillet heat and cook for 5 minutes. Add mushrooms and cook for 5 minutes. Add mushrooms and cook for 5 minutes. Add mushrooms and coo

In [ ]:
# ==========================================================
# 🍳 Gradio App for LoRA-Finetuned DistilGPT2 (Recipe Generator)
# ==========================================================

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import gradio as gr

# ---------- Paths ----------
BASE_MODEL = "distilgpt2"
CHECKPOINT_DIR = "/kaggle/working/gpt2_recipe_lora/checkpoint-2000"  # your LoRA adapter path
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ---------- Load Tokenizer ----------
print(">>> Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
special_tokens = {
    "pad_token": "<|pad|>",
    "bos_token": "<|startofrecipe|>",
    "eos_token": "<|endofrecipe|>"
}
tokenizer.add_special_tokens(special_tokens)

# ---------- Load Base Model ----------
print(">>> Loading base model...")
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)
base_model.resize_token_embeddings(len(tokenizer))

# ---------- Load LoRA Adapter ----------
print(">>> Loading LoRA fine-tuned adapter...")
try:
    model = PeftModel.from_pretrained(base_model, CHECKPOINT_DIR)
    print("✅ LoRA adapter loaded successfully!")
except Exception as e:
    print(f"⚠️ Adapter load issue: {e}")
    print("Retrying with ignore_mismatched_sizes=True ...")
    model = PeftModel.from_pretrained(base_model, CHECKPOINT_DIR, ignore_mismatched_sizes=True)

model = model.to(DEVICE)
model.eval()

print("✅ Model ready for generation!")

# ---------- Generation Function ----------
def generate_recipe(title, ingredients, max_length=200, temperature=0.8, top_p=0.9):
    prompt = f"<|startofrecipe|> Title: {title}\nIngredients: {ingredients}\nDirections:"
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(DEVICE)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            max_length=max_length,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    recipe = generated_text.replace(prompt, "").strip()
    return recipe

# ---------- Gradio Interface ----------
demo = gr.Interface(
    fn=generate_recipe,
    inputs=[
        gr.Textbox(label="Recipe Title", placeholder="Creamy Mushroom Pasta"),
        gr.Textbox(label="Ingredients (comma-separated)", placeholder="mushroom, cream, garlic, pasta"),
        gr.Slider(50, 400, value=200, step=10, label="Max Length"),
        gr.Slider(0.5, 1.2, value=0.8, step=0.05, label="Temperature"),
        gr.Slider(0.6, 1.0, value=0.9, step=0.05, label="Top-p"),
    ],
    outputs=gr.Textbox(label="Generated Recipe Directions"),
    title="🍳 LoRA-Finetuned Recipe Generator",
    description="Generate creative cooking directions using a LoRA-finetuned DistilGPT2 model.",
    examples=[
        ["Garlic Butter Shrimp", "shrimp, garlic, butter, lemon, parsley"],
        ["Pancakes", "flour, milk, egg, sugar, butter, baking powder"],
        ["Spaghetti Aglio e Olio", "spaghetti, garlic, olive oil, chili flakes, parsley"]
    ]
)

demo.launch(debug=True, share=True)


>>> Loading tokenizer...
>>> Loading base model...
>>> Loading LoRA fine-tuned adapter...
⚠️ Adapter load issue: Error(s) in loading state_dict for PeftModelForCausalLM:
	size mismatch for base_model.model.transformer.wte.weight: copying a param with shape torch.Size([50259, 768]) from checkpoint, the shape in current model is torch.Size([50260, 768]).
	size mismatch for base_model.model.lm_head.weight: copying a param with shape torch.Size([50259, 768]) from checkpoint, the shape in current model is torch.Size([50260, 768]).
Retrying with ignore_mismatched_sizes=True ...
✅ Model ready for generation!


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1803: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:477: UserWarning: Some weights of PeftModelForCausalLM were not initialized from the model checkpoint and are being ignored because you passed `ignore_mismatched_sizes=True`: - base_model.model.lm_head.weight: found shape torch.Size([50259, 768]) in the checkpoint and torch.Size([50260, 768]) in the model instantiated
- base_model.model.transformer.wte.weight: found shape torch.Size([50259, 768]) in the checkpoint and torch.Size([50260, 768]) in the model instantiated.
  warn

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://357c073fe9352b950f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
